# Workflow for updating the benchmark

This workflow adds texts to the benchmark and checks for duplicates.

## I. Consistency with the benchmark setup

* Check that LabelStudio files are properly named
* Check that LabelStudio files are properly labelled

In [1]:
import json

filename = 'labelled_data/substringtagger_labels/koond_1000_levinumad_labelled.json'

#check it can be opened and json loaded
with open(filename,'r',encoding='UTF-8') as f:
    data = json.load(f)

In [2]:
for task in data:
    #we have one annotation per text in this setup
    assert len(task['annotations']) == 1
    #check that the task has been annotated by someone
    assert task['annotations'][0]['completed_by'] > 0
    #check that yes or no has been chosen
    assert task['annotations'][0]['result'][-1]['value']['choices'][0] in ['yes','no']

## II. Update the benchmark data

* Compute the necessary description files
* Explain the update procedure to user: what files must be added to the benchmark
* How to check that there are no overlaps in the resulting benchmark